# Load Depenency

There is some json errror in the download data that need to be fixed before we can load the data into pandas dataframe.

In [1]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [2]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, min_rnk=1, max_studies=100):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [4]:
# disease=input()
# print(disease)

In [5]:
# disease = 'parkinson'

#disease = 'depression'

# disease = 'diabetes'

# disease = 'tacs'

disease = 'TMS'


hit mesh some day

Pull ONE

#Explore Data

In [6]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=2 ))


In [7]:
# type(infodf)

In [8]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

2339


In [9]:
count = ct.get_study_count(search_expr=disease)


In [10]:
# count = 3000
start = 0
end = count
iter = 99 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()

array = {}
array ['key'] = 'value'
json_data = json.dumps(array)

for i in range(1, count, iter):
    # print(temp)
    temp =ct.get_full_studies(
        search_expr=disease,
        max_studies=iter,
        min_rnk=i,
        
        # fmt="csv",
        )
    iter2 = iter + (i-1)
    print(str(i) + "-" + str(iter2) + " of " + str(count))
    # print(temp2[0])
    temp2 =  pd.json_normalize(temp)
    # json_object = temp['FullStudiesResponse']
    # array = temp['FullStudiesResponse']
    # temp2 = pd.json_normalize(temp())
    new_df = pd.concat([new_df, temp2])
 
# https://clinicaltrials.gov/api/gui/demo/simple_full_study

1-99 of 2339
100-198 of 2339
199-297 of 2339
298-396 of 2339
397-495 of 2339
496-594 of 2339
595-693 of 2339
694-792 of 2339
793-891 of 2339
892-990 of 2339
991-1089 of 2339
1090-1188 of 2339
1189-1287 of 2339
1288-1386 of 2339
1387-1485 of 2339
1486-1584 of 2339
1585-1683 of 2339
1684-1782 of 2339
1783-1881 of 2339
1882-1980 of 2339
1981-2079 of 2339
2080-2178 of 2339
2179-2277 of 2339
2278-2376 of 2339


In [11]:
print(type(new_df))

<class 'pandas.core.frame.DataFrame'>


In [12]:
new_df.to_excel(disease+'_data_json_raw.xlsx', index = False)

In [13]:
new_df

,FullStudiesResponse.APIVrs,FullStudiesResponse.DataVrs,FullStudiesResponse.Expression,FullStudiesResponse.NStudiesAvail,FullStudiesResponse.NStudiesFound,FullStudiesResponse.MinRank,FullStudiesResponse.MaxRank,FullStudiesResponse.NStudiesReturned,FullStudiesResponse.FullStudies
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,1,99,99,"[{'Rank': 1, 'Study': {'ProtocolSection': {'Id..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,100,198,99,"[{'Rank': 100, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,199,297,99,"[{'Rank': 199, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,298,396,99,"[{'Rank': 298, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,397,495,99,"[{'Rank': 397, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,496,594,99,"[{'Rank': 496, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,595,693,99,"[{'Rank': 595, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,694,792,99,"[{'Rank': 694, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,793,891,99,"[{'Rank': 793, 'Study': {'ProtocolSection': {'..."
0,1.01.05,2022:09:15 23:06:14.343,TMS,427974,2339,892,990,99,"[{'Rank': 892, 'Study': {'ProtocolSection': {'..."


In [14]:
df3 = (new_df['FullStudiesResponse.FullStudies'])



In [15]:
# # data = json.loads(new_df.decode())
# # df3 = new_df(json_normalize(data, 'FullStudiesResponse.FullStudies'))
# df3 = new_df.FullStudiesResponse(json_normalize)
# print(df)

In [16]:
df3

0    [{'Rank': 1, 'Study': {'ProtocolSection': {'Id...
0    [{'Rank': 100, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 199, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 298, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 397, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 496, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 595, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 694, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 793, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 892, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 991, 'Study': {'ProtocolSection': {'...
0    [{'Rank': 1090, 'Study': {'ProtocolSection': {...
0    [{'Rank': 1189, 'Study': {'ProtocolSection': {...
0    [{'Rank': 1288, 'Study': {'ProtocolSection': {...
0    [{'Rank': 1387, 'Study': {'ProtocolSection': {...
0    [{'Rank': 1486, 'Study': {'ProtocolSection': {...
0    [{'Rank': 1585, 'Study': {'ProtocolSection': {...
0    [{'Rank': 1684, 'Study': {'ProtocolSection': {...
0    [{'Ra

In [17]:
# with open('data2.json', 'w', encoding='utf-8') as f:
#     json.dump(df3, f, ensure_ascii=False, indent=4)

'NStudiesFound': 3656,

Save file

In [18]:
# result.to_csv(r'data_raw.csv', index = False)
new_df.to_excel(disease+'_full_json_data_raw.xlsx', index = False)

In [19]:
new_df.to_hdf(disease+'full_json_data_raw.h5', key='df', mode='w')  

/var/folders/7x/knvg10qn19740t2kqn0qwy_40000gn/T/ipykernel_35596/394239288.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['FullStudiesResponse.APIVrs', 'FullStudiesResponse.DataVrs',
       'FullStudiesResponse.Expression', 'FullStudiesResponse.FullStudies'],
      dtype='object')]

  new_df.to_hdf(disease+'full_json_data_raw.h5', key='df', mode='w')


In [20]:
# new_df.to_json(disease+'full_json_data_raw.json')

ValueError: DataFrame index must be unique for orient='columns'.

In [ ]:
# labels, levels = pd.factorize(result["Phase"])  

In [ ]:
# labels

In [ ]:
# fun_barplot()